In [ ]:
!pip install wandb
import wandb
wandb.init(project="DL-1", entity="mshoaibvohra", settings=wandb.Settings(init_timeout=200))


CommError: Run initialization has timed out after 200.0 sec. Please try increasing the timeout with the `init_timeout` setting: `wandb.init(settings=wandb.Settings(init_timeout=120))`.

In [ ]:
import os
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

# Predefined Taxonomies
taxonomy_first = {
    "URW": 0,  # War-related narratives
    "CC": 1,   # Climate change narratives
    "Other": 2  # Other narratives
}

taxonomy = {
    "Other": [],
    "Blaming the war on others rather than the invader": [
        "Ukraine is the aggressor",
        "The West are the aggressors",
    ],
    "Discrediting Ukraine": [
        "Rewriting Ukraine’s history",
        "Discrediting Ukrainian nation and society",
        "Discrediting Ukrainian military",
        "Discrediting Ukrainian government and officials and policies",
        "Ukraine is a puppet of the West",
        "Ukraine is a hub for criminal activities",
        "Ukraine is associated with nazism",
        "Situation in Ukraine is hopeless",
    ],
    "Russia is the Victim": [
        "The West is russophobic",
        "Russia actions in Ukraine are only self-defence",
        "UA is anti-RU extremists",
    ],
    "Praise of Russia": [
        "Praise of Russian military might",
        "Praise of Russian President Vladimir Putin",
        "Russia is a guarantor of peace and prosperity",
        "Russia has international support from a number of countries and people",
        "Russian invasion has strong national support",
    ],
    "Overpraising the West": [
        "NATO will destroy Russia",
        "The West belongs in the right side of history",
        "The West has the strongest international support",
    ],
    "Speculating war outcomes": [
        "Russian army is collapsing",
        "Russian army will lose all the occupied territories",
        "Ukrainian army is collapsing",
    ],
    "Discrediting the West, Diplomacy": [
        "The EU is divided",
        "The West is weak",
        "The West is overreacting",
        "The West does not care about Ukraine, only about its interests",
        "Diplomacy does/will not work",
        "West is tired of Ukraine",
    ],
    "Negative Consequences for the West": [
        "Sanctions imposed by Western countries will backfire",
        "The conflict will increase the Ukrainian refugee flows to Europe",
    ],
    "Distrust towards Media": [
        "Western media is an instrument of propaganda",
        "Ukrainian media cannot be trusted",
    ],
    "Amplifying war-related fears": [
        "By continuing the war we risk WWIII",
        "Russia will also attack other countries",
        "There is a real possibility that nuclear weapons will be employed",
        "NATO should/will directly intervene",
    ],
    "Hidden plots by secret schemes of powerful groups": [],
    "Criticism of climate policies": [
        "Climate policies are ineffective",
        "Climate policies have negative impact on the economy",
        "Climate policies are only for profit",
    ],
    "Criticism of institutions and authorities": [
        "Criticism of the EU",
        "Criticism of international entities",
        "Criticism of national governments",
        "Criticism of political organizations and figures",
    ],
    "Climate change is beneficial": [
        "CO2 is beneficial",
        "Temperature increase is beneficial",
    ],
    "Downplaying climate change": [
        "Climate cycles are natural",
        "Weather suggests the trend is global cooling",
        "Temperature increase does not have significant impact",
        "CO2 concentrations are too small to have an impact",
        "Human activities do not impact climate change",
        "Ice is not melting",
        "Sea levels are not rising",
        "Humans and nature will adapt to the changes",
    ],
    "Questioning the measurements and science": [
        "Methodologies/metrics used are unreliable/faulty",
        "Data shows no temperature increase",
        "Greenhouse effect/carbon dioxide do not drive climate change",
        "Scientific community is unreliable",
    ],
    "Criticism of climate movement": [
        "Climate movement is alarmist",
        "Climate movement is corrupt",
        "Ad hominem attacks on key activists",
    ],
    "Controversy about green technologies": [
        "Renewable energy is dangerous",
        "Renewable energy is unreliable",
        "Renewable energy is costly",
        "Nuclear energy is not climate friendly",
    ],
    "Amplifying Climate Fears": [
        "Earth will be uninhabitable soon",
        "Amplifying existing fears of global warming",
        "Doomsday scenarios for humans",
        "Whatever we do it is already too late",
    ],
    "Green policies are geopolitical instruments": [
        "Climate-related international relations are abusive/exploitative",
        "Green activities are a form of neo-colonialism",
    ],
}

In [11]:
import os
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score


subnarratives = {
    sub: idx for narrative, subs in taxonomy.items() for idx, sub in enumerate(subs)
}

# Helper Functions
def parse_annotations(annotation_file):
    data = []
    with open(annotation_file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) >= 3:
                file_name = parts[0]
                group_narrative_sub = parts[2].split(";")

                groups = set()
                narratives = set()
                subnarratives_list = set()

                for entry in group_narrative_sub:
                    levels = entry.split(":")
                    if len(levels) >= 1:
                        groups.add(levels[0].strip())
                    if len(levels) >= 2:
                        narratives.add(levels[1].strip())
                    if len(levels) == 3:
                        subnarratives_list.add(levels[2].strip())

                data.append({
                    "file_name": file_name,
                    "groups": list(groups),
                    "narratives": list(narratives),
                    "subnarratives": list(subnarratives_list),
                })
    return data

def encode_labels(data, level_mapping):
    labels = []
    for item in data:
        label_array = [0] * len(level_mapping)
        for label in item:
            if label in level_mapping:
                label_array[level_mapping[label]] = 1
        labels.append(label_array)
    return torch.tensor(labels, dtype=torch.float)

def load_and_encode_dataset(folder_path, level_mapping, level_key):
    annotation_file = os.path.join(folder_path, 'subtask-2-annotations.txt')
    data = parse_annotations(annotation_file)

    texts = []
    labels = []

    for item in data:
        file_path = os.path.join(folder_path, item['file_name'])
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                texts.append(f.read())
                labels.append(item[level_key])

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
    label_tensor = encode_labels(labels, level_mapping)

    return encodings, label_tensor

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

def compute_metrics(p):
    logits = p.predictions
    probabilities = torch.sigmoid(torch.tensor(logits)).numpy()
    preds = (probabilities >= 0.5).astype(int)
    labels = p.label_ids

    f1 = f1_score(labels, preds, average='micro')
    precision = precision_score(labels, preds, average='micro', zero_division=1)
    recall = recall_score(labels, preds, average='micro')

    return {'f1': f1, 'precision': precision, 'recall': recall}

# Filtering Functions
def filter_by_group(encodings, labels, selected_groups):
    mask = selected_groups.any(axis=1)  # Create a mask based on group selection
    filtered_encodings = {key: val[mask] for key, val in encodings.items()}
    filtered_labels = labels[mask]  # Filter labels using the same mask
    return filtered_encodings, filtered_labels

def filter_by_narrative(encodings, labels, selected_narratives):
    mask = selected_narratives.any(axis=1)  # Create a mask based on narrative selection
    filtered_encodings = {key: val[mask] for key, val in encodings.items()}
    filtered_labels = labels[mask]  # Filter labels using the same mask
    return filtered_encodings, filtered_labels


# Training Function
def train_model(encodings, labels, output_dir, num_labels):
    print (encodings['input_ids'])
    train_inputs, val_inputs, train_labels, val_labels = train_test_split(
        encodings['input_ids'], labels, test_size=0.2, random_state=42
    )
    print (encodings['attention_mask'])
    train_masks, val_masks = train_test_split(
        encodings['attention_mask'], test_size=0.2, random_state=42
    )

    train_encodings = {'input_ids': train_inputs, 'attention_mask': train_masks}
    val_encodings = {'input_ids': val_inputs, 'attention_mask': val_masks}

    train_dataset = CustomDataset(train_encodings, train_labels)
    val_dataset = CustomDataset(val_encodings, val_labels)

    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=num_labels,
        problem_type="multi_label_classification"
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        num_train_epochs=8,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        save_steps=500,
        save_total_limit=2,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer, model

# Hierarchical Training
def hierarchical_training_with_dependency(train_dir):
    print("Training Group-Level Classifier...")
    group_encodings, group_labels = load_and_encode_dataset(train_dir, taxonomy_first, 'groups')
    group_trainer, group_model = train_model(group_encodings, group_labels, './group_results', len(taxonomy_first))

    group_predictions = group_trainer.predict(CustomDataset(group_encodings, group_labels)).predictions
    selected_groups = (group_predictions >= 0.5).astype(int)

    print("Training Narrative-Level Classifier...")
    narrative_encodings, narrative_labels = load_and_encode_dataset(
        train_dir,
        {key: idx for idx, key in enumerate(taxonomy.keys())},
        'narratives'
    )
    narrative_filtered_encodings, narrative_filtered_labels = filter_by_group(
        narrative_encodings, narrative_labels, selected_groups
    )
    narrative_trainer, narrative_model = train_model(
        narrative_filtered_encodings, narrative_filtered_labels, './narrative_results', len(taxonomy)
    )

    narrative_predictions = narrative_trainer.predict(
        CustomDataset(narrative_encodings, narrative_labels)
    ).predictions
    selected_narratives = (narrative_predictions >= 0.5).astype(int)

    print("Training Subnarrative-Level Classifier...")
    subnarrative_encodings, subnarrative_labels = load_and_encode_dataset(train_dir, subnarratives, 'subnarratives')
    subnarrative_filtered_encodings, subnarrative_filtered_labels = filter_by_narrative(
        subnarrative_encodings, subnarrative_labels, selected_narratives
    )
    train_model(
        subnarrative_filtered_encodings, subnarrative_filtered_labels, './subnarrative_results', len(subnarratives)
    )


# Execute Hierarchical Training
hierarchical_training_with_dependency('train')


Training Group-Level Classifier...
tensor([[  101,  1996,  2088,  ...,  1996,  4736,   102],
        [  101, 15561,  1998,  ...,  9236,  8496,   102],
        [  101,  5924,  1005,  ...,  6707,  1997,   102],
        ...,
        [  101,  3422,  9788,  ..., 22035,  2102,   102],
        [  101,  3607, 26960,  ...,  2711,  2001,   102],
        [  101,  9437,  6507,  ...,  2348,  4501,   102]])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training Narrative-Level Classifier...
tensor([[  101,  1996,  2088,  ...,  1996,  4736,   102],
        [  101, 15561,  1998,  ...,  9236,  8496,   102],
        [  101,  5924,  1005,  ...,  6707,  1997,   102],
        ...,
        [  101,  3422,  9788,  ..., 22035,  2102,   102],
        [  101,  3607, 26960,  ...,  2711,  2001,   102],
        [  101,  9437,  6507,  ...,  2348,  4501,   102]])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training Subnarrative-Level Classifier...
tensor([[  101,  5924,  1005,  ...,  6707,  1997,   102],
        [  101,  1996,  3663,  ...,  1998,  2903,   102],
        [  101, 27838,  7770,  ...,  2007,  3607,   102],
        ...,
        [  101,  3607,  2038,  ...,  8635,  2106,   102],
        [  101,  5842,  2000,  ..., 24481, 11768,   102],
        [  101, 22072,  1024,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification
# import os

# def get_latest_checkpoint(folder):
#     checkpoints = [d for d in os.listdir(folder) if d.startswith('checkpoint')]
#     checkpoints.sort(key=lambda x: int(x.split('-')[-1]))
#     return os.path.join(folder, checkpoints[-1])

# def predict_test_files(test_dir, output_file):
#     # Load trained models


#     group_model_path = get_latest_checkpoint('./group_results')
#     narrative_model_path = get_latest_checkpoint('./narrative_results')
#     subnarrative_model_path = get_latest_checkpoint('./subnarrative_results')

#     group_model = BertForSequenceClassification.from_pretrained(group_model_path)
#     narrative_model = BertForSequenceClassification.from_pretrained(narrative_model_path)
#     subnarrative_model = BertForSequenceClassification.from_pretrained(subnarrative_model_path)

#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#     test_files = [f for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]
#     results = []

#     for test_file in test_files:
#         file_path = os.path.join(test_dir, test_file)

#         with open(file_path, 'r') as f:
#             text = f.read()

#         # Tokenize the text
#         encoding = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors="pt")

#         # Group Predictions
#         group_logits = group_model(**{key: val.to(group_model.device) for key, val in encoding.items()}).logits
#         group_probs = torch.sigmoid(group_logits).cpu().detach().numpy()
#         group_preds = (group_probs >= 0.5).astype(int)

#         # Narrative Predictions
#         narrative_logits = narrative_model(**{key: val.to(narrative_model.device) for key, val in encoding.items()}).logits
#         narrative_probs = torch.sigmoid(narrative_logits).cpu().detach().numpy()
#         narrative_preds = (narrative_probs >= 0.5).astype(int)

#         # Subnarrative Predictions
#         subnarrative_logits = subnarrative_model(**{key: val.to(subnarrative_model.device) for key, val in encoding.items()}).logits
#         subnarrative_probs = torch.sigmoid(subnarrative_logits).cpu().detach().numpy()
#         subnarrative_preds = (subnarrative_probs >= 0.5).astype(int)

#         # Collect results
#         results.append({
#             "file": test_file,
#             "groups": [key for idx, key in enumerate(taxonomy_first.keys()) if group_preds[0][idx] == 1],
#             "narratives": [key for idx, key in enumerate(taxonomy.keys()) if narrative_preds[0][idx] == 1],
#             "subnarratives": [key for idx, key in enumerate(subnarratives.keys()) if subnarrative_preds[0][idx] == 1]
#         })

#     # Save results to output.txt
#     with open(output_file, 'w') as f:
#         for result in results:
#             f.write(f"File: {result['file']}\n")
#             f.write(f"Groups: {', '.join(result['groups'])}\n")
#             f.write(f"Narratives: {', '.join(result['narratives'])}\n")
#             f.write(f"Subnarratives: {', '.join(result['subnarratives'])}\n")
#             f.write("\n")

#     # Print results to console
#     for result in results:
#         print(f"File: {result['file']}")
#         print(f"Groups: {', '.join(result['groups'])}")
#         print(f"Narratives: {', '.join(result['narratives'])}")
#         print(f"Subnarratives: {', '.join(result['subnarratives'])}")
#         print()

# # Run prediction
# predict_test_files('test', 'output.txt')


FileNotFoundError: [Errno 2] No such file or directory: './group_results'

In [14]:
def get_latest_checkpoint(folder):
    checkpoints = [d for d in os.listdir(folder) if d.startswith('checkpoint')]
    checkpoints.sort(key=lambda x: int(x.split('-')[-1]))
    return os.path.join(folder, checkpoints[-1])

In [22]:
def hierarchical_testing_with_dependency(test_dir, output_file):
    # Load trained models
    group_model_path = get_latest_checkpoint('./group_results')
    narrative_model_path = get_latest_checkpoint('./narrative_results')
    subnarrative_model_path = get_latest_checkpoint('./subnarrative_results')

    group_model = BertForSequenceClassification.from_pretrained(group_model_path)
    narrative_model = BertForSequenceClassification.from_pretrained(narrative_model_path)
    subnarrative_model = BertForSequenceClassification.from_pretrained(subnarrative_model_path)

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Define group-to-narrative mapping
    group_to_narrative = {
        "URW": [key for key in taxonomy.keys() if key not in ["Criticism of climate movement", "Downplaying climate change"]],
        "CC": [key for key in taxonomy.keys() if "climate" in key.lower()],
        "Other": []  # No specific narratives for "Other"
    }

    # Parse test files
    test_files = [f for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))]
    results = []

    for test_file in test_files:
        file_path = os.path.join(test_dir, test_file)

        with open(file_path, 'r') as f:
            text = f.read()

        # Tokenize the text
        encoding = tokenizer(text, truncation=True, padding=True, max_length=512, return_tensors="pt")

        # Step 1: Predict Group
        group_logits = group_model(**{key: val.to(group_model.device) for key, val in encoding.items()}).logits
        group_probs = torch.sigmoid(group_logits).cpu().detach().numpy()
        group_preds = (group_probs >= 0.5).astype(int)

        selected_groups = [
            group for idx, group in enumerate(taxonomy_first.keys()) if group_preds[0][idx] == 1
        ]

        if "Other" in selected_groups:
            # If "Other" is the predicted group, no further filtering is needed
            results.append({
                "file": test_file,
                "narratives": ["Other"],
                "subnarratives": ["Other"]
            })
            continue

        # Filter narratives based on selected groups
        filtered_narratives = [
            (group, narrative)
            for group in selected_groups
            for narrative in group_to_narrative.get(group, [])
        ]

        # Step 2: Predict Narratives
        narrative_encodings = {key: val.to(narrative_model.device) for key, val in encoding.items()}
        narrative_logits = narrative_model(**narrative_encodings).logits
        narrative_probs = torch.sigmoid(narrative_logits).cpu().detach().numpy()
        narrative_preds = (narrative_probs >= 0.5).astype(int)

        selected_narratives = [
            (group, narrative)
            for idx, (group, narrative) in enumerate(filtered_narratives) if narrative_preds[0][idx] == 1
        ]

        # Step 3: Predict Subnarratives
        filtered_subnarratives = [
            sub for _, narrative in selected_narratives for sub in taxonomy.get(narrative, [])
        ]

        subnarrative_encodings = {key: val.to(subnarrative_model.device) for key, val in encoding.items()}
        subnarrative_logits = subnarrative_model(**subnarrative_encodings).logits
        subnarrative_probs = torch.sigmoid(subnarrative_logits).cpu().detach().numpy()
        subnarrative_preds = (subnarrative_probs >= 0.5).astype(int)

        selected_subnarratives = [
            sub for idx, sub in enumerate(filtered_subnarratives) if subnarrative_preds[0][idx] == 1
        ]

        # Format narratives with group prefix
        narratives_with_groups = [
            f"{group}:{narrative}" for group, narrative in selected_narratives
        ]

        # Collect results
        results.append({
            "file": test_file,
            "narratives": narratives_with_groups,
            "subnarratives": selected_subnarratives
        })

    # Save results to output file
    with open(output_file, 'w') as f:
        for result in results:
            narratives = ";".join(result["narratives"])
            subnarratives = ";".join(result["subnarratives"])
            f.write(f"{result['file']}\t{narratives}\t{subnarratives}\n")

    # Print the output file content
    with open(output_file, 'r') as f:
        print(f.read())


hierarchical_testing_with_dependency('test', 'output.txt')


EN_CC_200049.txt		
EN_CC_200079.txt		
EN_UA_DEV_100029.txt	URW:Discrediting the West, Diplomacy	The EU is divided;The West is weak;The West is overreacting
EN_UA_DEV_215.txt	URW:Discrediting the West, Diplomacy	The EU is divided;The West is weak;The West is overreacting
EN_UA_DEV_100034.txt		
EN_CC_200085.txt	Other	Other
EN_CC_200047.txt		
EN_CC_200035.txt	Other	Other
EN_CC_200077.txt		
EN_UA_DEV_100003.txt	URW:Discrediting the West, Diplomacy	The EU is divided;The West is weak;The West is overreacting
EN_UA_DEV_100004.txt		
EN_CC_200069.txt		
EN_CC_200078.txt	Other	Other
EN_CC_200036.txt		
EN_CC_200046.txt		
EN_UA_DEV_213.txt	URW:Discrediting the West, Diplomacy;URW:Amplifying war-related fears	The EU is divided;The West is weak;The West is overreacting
EN_CC_200064.txt	Other	Other
EN_UA_DEV_214.txt	URW:Discrediting the West, Diplomacy;URW:Amplifying war-related fears	The EU is divided;The West is weak;The West is overreacting
EN_UA_DEV_100036.txt		
EN_CC_200050.txt		
EN_UA_DEV_20.txt